# Deploy DeepSeek  R1 Models on Amazon Bedrock using Custom Model Imports

## DeepSeek-R1-Distill models are fine-tuned based on open-source models, using samples generated by DeepSeek-R1. 

Out of 6 models, following two are based on Llama architecture. 
- DeepSeek-R1-Distill-Llama-8B 
- Llama-3.3-70B-Instruct 

In this notebook we will deploy models based on Llama architecture

### Install required Dependencies

In [ ]:
%pip install huggingface_hub ipywidgets boto3
!jupyter nbextension enable --py widgetsnbextension

In [28]:
import os
from huggingface_hub import snapshot_download

# Get current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

import ipywidgets as widgets
from IPython.display import display
import os

# Define available models
available_models = {
    "DeepSeek-R1-Distill-Llama-8B": "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "DeepSeek-R1-Distill-Llama-70B": "deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
}

# Create global variables for selected model
global selected_model_name
global selected_model_id
selected_model_name = "DeepSeek-R1-Distill-Llama-8B"
selected_model_id = available_models[selected_model_name]

# Create model dropdown
model_dropdown = widgets.Dropdown(
    options=list(available_models.keys()),
    value=selected_model_name,
    description='Model:',
    style={'description_width': 'initial'},
    layout={'width': 'auto'}
)

# Create select button
select_button = widgets.Button(
    description='Select Model',
    style={'description_width': 'initial'},
    button_style='success',
    layout={'width': 'auto'}
)

# Create output widget for status messages
output = widgets.Output()

def on_select_button_click(b):
    global selected_model_name
    global selected_model_id
    with output:
        output.clear_output()
        selected_model_name = model_dropdown.value
        selected_model_id = available_models[selected_model_name]
        print(f"Selected model: {selected_model_name}")
        print(f"Model ID: {selected_model_id}")

select_button.on_click(on_select_button_click)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>Select Model</h3>"),
    model_dropdown,
    select_button,
    output
]))


Current directory: d:\DeepSeek


In [30]:
# Model ID picked up from the cell above, 
model_id = selected_model_id

try:
    # Download the model
    local_dir = snapshot_download(
        repo_id=model_id,
        local_dir=os.path.join(current_dir, selected_model_name),
        ignore_patterns=["*.md", "*.h5"],  
        local_dir_use_symlinks=False 
    )
    print(f"Model downloaded successfully to: {local_dir}")
except Exception as e:
    print(f"An error occurred during download: {e}")



Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Model downloaded successfully to: D:\DeepSeek\DeepSeek-R1-Distill-Llama-8B


In [25]:
import boto3
import ipywidgets as widgets
from IPython.display import display
import os

# Set default region
default_region = 'us-east-1'
boto3.setup_default_session(region_name=default_region)

# Create a global variable to store the selected region
global selected_region
selected_region = default_region

def get_available_regions():
    """Get list of available AWS regions"""
    ec2_client = boto3.client('ec2', region_name=default_region)
    regions = [region['RegionName'] for region in ec2_client.describe_regions()['Regions']]
    return sorted(regions)

# Create region dropdown
region_dropdown = widgets.Dropdown(
    options=get_available_regions(),
    value=default_region,
    description='Region:',
    style={'description_width': 'initial'},
    layout={'width': 'auto'}
)

# Create select button
select_button = widgets.Button(
    description='Select Region',
    style={'description_width': 'initial'},
    button_style='success',
    layout={'width': 'auto'}
)

# Create output widget for status messages
output = widgets.Output()

def on_select_button_click(b):
    global selected_region
    with output:
        output.clear_output()
        selected_region = region_dropdown.value
        print(f"Selected region: {selected_region}")
        # Update boto3 session with new region
        boto3.setup_default_session(region_name=selected_region)

select_button.on_click(on_select_button_click)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>Select AWS Region</h3>"),
    region_dropdown,
    select_button,
    output
]))


In [26]:
def get_buckets_in_region(region):
    """Get list of S3 buckets in the specified region"""
    s3_client = boto3.client('s3', region_name=region)
    buckets = []
    try:
        response = s3_client.list_buckets()
        for bucket in response['Buckets']:
            try:
                bucket_region = selected_region
                # Note: us-east-1 returns None instead of 'us-east-1'
                if bucket_region is None:
                    bucket_region = 'us-east-1'
                if bucket_region == region:
                    buckets.append(bucket['Name'])
            except Exception as e:
                print(f"Error getting location for bucket {bucket['Name']}: {e}")
                continue
    except Exception as e:
        print(f"Error listing buckets: {e}")
    return sorted(buckets)

def upload_directory_to_s3(local_directory, bucket_name, s3_prefix=""):
    """
    Upload a directory to S3
    :param local_directory: Local directory path
    :param bucket_name: S3 bucket name
    :param s3_prefix: Prefix to add to S3 keys (optional)
    """
    s3_client = boto3.client('s3', region_name=selected_region)
    
    # Walk through the local directory
    for root, dirs, files in os.walk(local_directory):
        for file in files:
            # Get the full local path
            local_path = os.path.join(root, file)
            
            # Calculate relative path for S3 key
            relative_path = os.path.relpath(local_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path).replace("\\", "/")
            
            try:
                print(f"Uploading {local_path} to s3://{bucket_name}/{s3_key}")
                s3_client.upload_file(local_path, bucket_name, s3_key)
            except Exception as e:
                print(f"Error uploading {local_path}: {e}")

# Create bucket dropdown
bucket_dropdown = widgets.Dropdown(
    options=get_buckets_in_region(selected_region),
    description='S3 Bucket:',
    style={'description_width': 'initial'},
    layout={'width': 'auto'}
)

# Create refresh button for buckets
refresh_button = widgets.Button(
    description='Refresh Buckets',
    style={'description_width': 'initial'},
    button_style='info'
)

def on_refresh_click(b):
    bucket_dropdown.options = get_buckets_in_region(selected_region)
    bucket_dropdown.value = None

refresh_button.on_click(on_refresh_click)

# Create upload button
upload_button = widgets.Button(
    description='Upload to S3',
    style={'description_width': 'initial'},
    button_style='primary'
)

def on_upload_button_click(b):
    if not bucket_dropdown.value:
        print("Please select a bucket first!")
        return
    
    print(f"Starting upload to bucket: {bucket_dropdown.value}")
    try:
        upload_directory_to_s3(
            local_directory=os.path.join(current_dir, "DeepSeek-R1-Distill-Llama-8B"),
            bucket_name=bucket_dropdown.value,
            s3_prefix="DeepSeek-R1-Distill-Llama-8B"
        )
        print("Upload completed successfully!")
    except Exception as e:
        print(f"Error during upload: {e}")

upload_button.on_click(on_upload_button_click)

# Display bucket selection and upload controls
display(widgets.VBox([
    widgets.HTML("<h3>Select S3 Bucket and Upload</h3>"),
    bucket_dropdown,
    widgets.HBox([refresh_button, upload_button])
]))


In [27]:
import json
import boto3
from botocore.config import Config


REGION_NAME = selected_region
MODEL_ID= 'arn:aws:bedrock:us-east-1:{youraccountnumber}:imported-model/{modelId}}'

config = Config(
    retries={
        'total_max_attempts': 10, 
        'mode': 'standard'
    }
)
message = "What is the color of the sky?"


session = boto3.session.Session()
br_runtime = session.client(service_name = 'bedrock-runtime', 
                                 region_name=REGION_NAME, 
                                 config=config)
    
try:
    invoke_response = br_runtime.invoke_model(modelId=MODEL_ID, 
                                            body=json.dumps({'prompt': message}), 
                                            accept="application/json", 
                                            contentType="application/json")
    invoke_response["body"] = json.loads(invoke_response["body"].read().decode("utf-8"))
    print(json.dumps(invoke_response, indent=4))
except Exception as e:
    print(e)
    print(e.__repr__())
        
    

{
    "ResponseMetadata": {
        "RequestId": "00a18367-552c-413d-90ea-2a05fbf52279",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "date": "Mon, 27 Jan 2025 19:43:52 GMT",
            "content-type": "application/json",
            "content-length": "2607",
            "connection": "keep-alive",
            "x-amzn-requestid": "00a18367-552c-413d-90ea-2a05fbf52279",
            "x-amzn-bedrock-invocation-latency": "8940",
            "x-amzn-bedrock-output-token-count": "512",
            "x-amzn-bedrock-input-token-count": "9"
        },
        "RetryAttempts": 0
    },
    "contentType": "application/json",
    "body": {
        "generation": " It's a question that might seem simple, but the answer isn't straightforward. The sky appears blue during the day and darkens to black at night. But what causes this? Let's explore the science behind it.\n\nFirst, during the day, the sky is blue because of the scattering of sunlight. Sunlight consists of various col